In [1]:
from sort.sort import Sort

mot_tracker = Sort()

In [2]:
from ultralytics import YOLO
import cv2
import numpy as np

In [3]:
from util import get_car, read_license_plate, write_csv

Using CPU. Note: This module is much faster with a GPU.


### Load models

In [4]:
coco_model = YOLO('./models/yolov8n.pt')
license_plate_detector = YOLO('./models/license_plate_detector.pt')

### Load video

In [5]:
capture = cv2.VideoCapture('./data/sample.mp4')

### Read frames

In [6]:
results = {}

vehicles = [2, 3, 5, 7]

# read frames
frame_nmr = -1
ret = True
while ret:
    frame_nmr += 1
    ret, frame = capture.read()
    if ret and frame_nmr < 3: # I will take just the first 3 frames to test ...
        results[frame_nmr] = {}
        # detect vehicles
        detections = coco_model(frame)[0]
        detections_ = []
        for detection in detections.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = detection
            if int(class_id) in vehicles:
                detections_.append([x1, y1, x2, y2, score])

        # track vehicles
        track_ids = mot_tracker.update(np.asarray(detections_))

        # detect license plates
        license_plates = license_plate_detector(frame)[0]
        for license_plate in license_plates.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = license_plate

            # assign license plate to car
            xcar1, ycar1, xcar2, ycar2, car_id = get_car(license_plate, track_ids)

            if car_id != -1:

                # crop license plate
                license_plate_crop = frame[int(y1):int(y2), int(x1): int(x2), :]

                # process license plate
                license_plate_crop_gray = cv2.cvtColor(license_plate_crop, cv2.COLOR_BGR2GRAY)
                _, license_plate_crop_thresh = cv2.threshold(license_plate_crop_gray, 64, 255, cv2.THRESH_BINARY_INV)

                # read license plate number
                license_plate_text, license_plate_text_score = read_license_plate(license_plate_crop_thresh)

                if license_plate_text is not None:
                    results[frame_nmr][car_id] = {'car': {'bbox': [xcar1, ycar1, xcar2, ycar2]},
                                                  'license_plate': {'bbox': [x1, y1, x2, y2],
                                                                    'text': license_plate_text,
                                                                    'bbox_score': score,
                                                                    'text_score': license_plate_text_score}}

# write results
write_csv(results, './data/test.csv')


0: 384x640 25 cars, 1 bus, 2 trucks, 64.0ms
Speed: 15.6ms preprocess, 64.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 license_plates, 66.4ms
Speed: 0.0ms preprocess, 66.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 25 cars, 1 bus, 2 trucks, 59.2ms
Speed: 0.0ms preprocess, 59.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 license_plates, 56.5ms
Speed: 2.0ms preprocess, 56.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 25 cars, 1 bus, 71.1ms
Speed: 0.0ms preprocess, 71.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 license_plates, 56.0ms
Speed: 0.0ms preprocess, 56.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
{'car': {'bbox': [902.8524169921875, 1410.2977294921875, 1520.022705078125, 1936.4622802734375]}, 'license_plate': {'bbox': [1104.6693115234375, 1723.4132080078125, 1296.2958984375,